# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [2]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [3]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
Enhance Investigation Resolution: Integrating Machine Learning and Core Scanning Techniques	Talk	talk-8	Kochi Core Center	2025-08-22	Kochi, Japan
Translating XRF into Various Measurements with an Inclusive Self-Supervised Model	Talk	talk-7	National Institute of Polar Research	2025-02-20	Tokyo, Japan
Machine learning and XRF core scanning: breaking barriers between projects	Talk	talk-6	Royal Netherlands Institute for Sea Research (NIOZ)	2024-10-08	Texel, Netherlands
Pretraining Foundation Models: Unleashing the Power of Forgotten Spectra for Various Geological Applications	Talk	talk-5	University of Bremen, Institute of Geography	2024-04-10	Bremen, Germany
Scale up geological research capacity: Machine learning and sediment core scanning techniques	Talk	talk-4	Kochi Core Center	2023-08-28	Kochi, Japan
Applying a Machine Learning Approach to Sediment-Facies Classification of Coastal Sediments from Northern Germany	Conference Tal

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [4]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

title             type  \
0  Enhance Investigation Resolution: Integrating ...             Talk   
1  Translating XRF into Various Measurements with...             Talk   
2  Machine learning and XRF core scanning: breaki...             Talk   
3  Pretraining Foundation Models: Unleashing the ...             Talk   
4  Scale up geological research capacity: Machine...             Talk   
5  Applying a Machine Learning Approach to Sedime...  Conference Talk   
6  Artificial intelligence for facies classificat...  Conference Talk   
7  From wiggles to statistics - a few thoughts an...             Talk   

  url_slug                                              venue        date  \
0   talk-8                                  Kochi Core Center  2025-08-22   
1   talk-7               National Institute of Polar Research  2025-02-20   
2   talk-6  Royal Netherlands Institute for Sea Research (...  2024-10-08   
3   talk-5       University of Bremen, Institute of Geography  2024-04-10   
4   talk-4                                  Kochi Core Center  2023-08-28   
5   talk-3            American Geophysical Union Fall Meeting  2021-12-14   
6   talk-2                     Japan Geoscience Union Meeting  2021-06-03   
7   talk-1      University of Innsbruck, Institute of Geology  2019-07-20   

             location  talk_url  description  
0        Kochi, Japan       NaN          NaN  
1        Tokyo, Japan       NaN          NaN  
2  Texel, Netherlands       NaN          NaN  
3     Bremen, Germany       NaN          NaN  
4        Kochi, Japan       NaN          NaN  
5  New Orleans, USA         NaN          NaN  
6        Chiba, Japan       NaN          NaN  
7  Innsbruck, Austria       NaN          NaN

## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [5]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [6]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [7]:
!ls ../_talks

2019-07-20-talk-1.md  2023-08-28-talk-4.md  2025-02-20-talk-7.md  talkmap.ipynb
2021-06-03-talk-2.md  2024-04-10-talk-5.md  2025-08-22-talk-7.md  talkmap.py
2021-12-14-talk-3.md  2024-10-08-talk-6.md  2025-08-22-talk-8.md


In [23]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
